In [6]:
import sys
sys.path.append("../v3-polars")

%load_ext autoreload
%autoreload 2

from v3 import state
import numpy as np

import polars as pl

from datetime import date, timedelta, datetime, timezone
import math
import os

from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
WETH = ['0x82aF49447D8a07e3bd95BD0d56f35241523fBab1'.lower(), 
        '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2'.lower()]

In [8]:
starting = datetime(year = 2023, month = 3, day = 1)
ending = datetime(year = 2024, month = 1, day = 1)

address = '0xc31e54c7a869b9fcbecc14363cf510d1c41fa443'
arb = state.v3Pool(address, 'arbitrum')#, update = True)
priceArb = arb.getPriceSeries(starting, frequency = '15m', gas = True)

address = '0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640'
eth = state.v3Pool(address, 'ethereum')#, update = True)
priceEth = eth.getPriceSeries(starting, frequency = '15m', gas = True)


PARTITIONED DS: estimated cardinality: 0.9995722 exceeded the boundary: 0.4, running default HASH AGGREGATION
PARTITIONED DS: estimated cardinality: 0.9964276 exceeded the boundary: 0.4, running default HASH AGGREGATION
PARTITIONED DS: estimated cardinality: 0.9987539 exceeded the boundary: 0.4, running default HASH AGGREGATION
PARTITIONED DS: estimated cardinality: 1 exceeded the boundary: 0.4, running default HASH AGGREGATION


In [20]:
def tokenNotWETH(pool, WETH):
    if pool.token0 in WETH:
        USDC = pool.token1
    elif pool.token1 in WETH:
        USDC = pool.token0
    else:
        raise ValueError("Token missing from WETH array")
        
    return USDC

def verbose_print(string, verbose):
    if verbose:
        print(string)
        
def binary_search_calldata(guessPrev, guessHigh, l1Calldata, l2Calldata, 
                          top_depth = 50, verbose = True):
    depth = 0
    search = True
    largest_spread = 1e12
    
    while search:     
        guessMid = (guessPrev + guessHigh) / 2
        
        l1Calldata['swapIn'] = guessMid
        l2Calldata['swapIn'] = guessMid

        l1AmtOut, l1Values = eth.swapIn(l1Calldata)
        l2AmtOut, l2Values = arb.swapIn(l2Calldata)
        
        l2AdjAmtOut = l2AmtOut - l2Gas
        l1AdjAmtOut = l1AmtOut - l1Gas
        spr = (l1AdjAmtOut - l2AdjAmtOut)
        
        (firstL1, secondL1, _) = l1Values
        
        (firstL2, secondL2, _) = l2Values
        if spr < -1 * largest_spread:
            verbose_print(f"Search low - New grid of {guessMid} to {guessHigh}", verbose)
            
            intPartMid = math.modf(guessMid)[1]
            intPartHigh = math.modf(guessHigh)[1]
            if intPartMid == intPartHigh:
                return (guessMid, l1Gas, l2Gas, l1AmtOut, l2AmtOut, (firstL1, secondL1), (firstL2, secondL2))
            
            guessPrev, guessHigh = guessMid, guessHigh

        elif spr > largest_spread:
            verbose_print(f"Search high - New grid of {guessPrev} to {guessMid}", verbose)
            
            intPartMid = math.modf(guessMid)[1]
            intPartPrev = math.modf(guessPrev)[1]
            if intPartMid == intPartPrev:
                return (guessMid, l1Gas, l2Gas, l1AmtOut, l2AmtOut, (firstL1, secondL1), (firstL2, secondL2))
            
            guessPrev, guessHigh = guessPrev, guessMid
            
        else:
            search = False
            verbose_print(f"Found price at {guessMid} with spr {spr}", verbose)
            (firstL1, secondL1, _) = l1Values
            (firstL2, secondL2, _) = l2Values
            
            return (guessMid, l1Gas, l2Gas, l1AmtOut, l2AmtOut, (firstL1, secondL1), (firstL2, secondL2))
        
        depth+=1
        if depth > top_depth:
            raise ValueError("Reached maximum depth")

In [27]:
data = []

l1Calldata = {'as_of': 0,
            'tokenIn': tokenNotWETH(eth, WETH),
            'swapIn': 0,
            'findMax': False,
             'forceHardShift': True}

    
l2Calldata = {'as_of': 0,
            'tokenIn': tokenNotWETH(arb, WETH),
            'swapIn': 0,
            'findMax': False,
             'forceHardShift': True}


medianL2Gas = priceArb['gas_price'].median()
medianL2GasUsed = priceArb['gas_used'].median()

medianL1Gas = priceEth['gas_price'].median()
medianL1GasUsed = priceEth['gas_used'].median()

iterator = [*priceArb.iter_rows()] #unpack that shit
for (ts, l2BN, _, gas_price, gas_used) in tqdm(iterator):
    startingAmtIn = 1e6
    ethRow = priceEth.filter(pl.col('block_timestamp') == ts)
    
    if ts >= pd.to_datetime("2024-01-01").to_pydatetime().replace(tzinfo = timezone.utc):
        continue

    if ethRow.is_empty():
        continue
        
    if gas_price == None:
        continue
    # -----

    l2Gas = medianL2Gas * medianL2GasUsed

    l2Calldata['as_of'] = l2BN
    l2Calldata['swapIn'] = startingAmtIn

    try:
        l2AmtOut, l2Values = arb.swapIn(l2Calldata)
    except Exception as e:
        print(f"Hit exception on L2 {e}")
        continue 
    # -----

    l1BN = ethRow['block_number'].item()
    l1Gas = (medianL1Gas * medianL1GasUsed)#.item()

    l1Calldata['as_of'] = l1BN
    l1Calldata['swapIn'] = startingAmtIn

    try:
        l1AmtOut, l1Values = eth.swapIn(l1Calldata)
    except Exception as e:
        print(f"Hit exception on L1 {e}")
        continue

    # -----

    l2AdjAmtOut = l2AmtOut - l2Gas
    l1AdjAmtOut = l1AmtOut - l1Gas

    if l2AdjAmtOut < l1AdjAmtOut:
        print(f'No value greater at {ts}')
        continue

    # search for the bounds
    searching = True
    while searching:
        prevAmtIn = startingAmtIn
        startingAmtIn *= 1.5
        l1Calldata['swapIn'] = startingAmtIn
        l2Calldata['swapIn'] = startingAmtIn

        l1AmtOut, _ = eth.swapIn(l1Calldata)
        l2AmtOut, _ = arb.swapIn(l2Calldata)

        l2AdjAmtOut = l2AmtOut - l2Gas
        l1AdjAmtOut = l1AmtOut - l1Gas

        if l1AdjAmtOut > l2AdjAmtOut:
             searching = False


    (amtOut, l1Gas, l2Gas, l1AmtOut, l2AmtOut, l1Values, l2Values) = binary_search_calldata(prevAmtIn, startingAmtIn, l1Calldata, l2Calldata, verbose = False)
    
    # send it
    data.append([ts, amtOut, l1Gas, l2Gas, l1AmtOut, l2AmtOut, l1Values, l2Values])

100%|███████████████████████████████████| 35447/35447 [7:40:21<00:00,  1.28it/s]


In [28]:
data_cleaned = []
for row in data:
    entry = []
    entry.extend([row[i] for i in range(0, 5 + 1)])
        
    (l1Before, l1After) = row[6]
    (l2Before, l2After) = row[7]
    entry.append(l1Before)
    entry.append(l1After)
    entry.append(l2Before)
    entry.append(l2After)
    
    data_cleaned.append(entry)

In [29]:
path = "data/arb_breakeven_setgas.csv"
df = (pl.DataFrame(data_cleaned, schema = ['date', 'cost', 'l1Gas', 'l2Gas', 'l1AmtOut', 
                                   'l2AmtOut', 'l1Before', 'l1After', 'l2Before', 'l2After'])
      .with_columns(chain = pl.lit('arb'))
     )

df.write_csv(path)

In [30]:
df = (pl.read_csv(path)
      .with_columns(l1PriceImpact = (pl.col('l1Before') ** 2 - pl.col('l1After') ** 2) / pl.col('l1After') ** 2,
                    l2PriceImpact = (pl.col('l2Before') ** 2 - pl.col('l2After') ** 2) / pl.col('l2After') ** 2,
                    gasCostDifference = (pl.col('l1Gas') - pl.col('l2Gas')) ,
                    ethPrice = (1 / (pl.col('l1Before') ** 2) * 1e12),
                    amtInDifference = (pl.col('l1AmtOut') - pl.col('l2AmtOut')))
      .with_columns(priceImpactSpr = -1 * (pl.col('l1PriceImpact') - pl.col('l2PriceImpact')) / pl.col('l2PriceImpact'))
     )

In [31]:
median_cost = df.select(pl.median('cost')).item() / 1e6
print(f'{median_cost:.2f}$')

68172.86$


In [32]:
costs = (
        df
        .with_columns(adjCost = pl.col('cost') / 1e6 / 1e3,
                      adjAmtInDifference = pl.col('amtInDifference') / 1e18 * (1 / (pl.col('l1Before') ** 2) * 1e12))
        .select('date', 'adjCost', 'adjAmtInDifference', 'priceImpactSpr', 'gasCostDifference')
        .with_columns(date = pl.col("date").str.to_datetime())
        .sort('date')
        .group_by_dynamic("date", every = '1d')
        .agg(pl.col('adjCost').quantile(.5),
             pl.col('adjAmtInDifference').quantile(.5),
             pl.col('priceImpactSpr').quantile(.5),
             pl.col('gasCostDifference').quantile(.5))
        .with_columns(adjCost=pl.col("adjCost").rolling_mean(window_size=7),
                      adjAmtInDifference=pl.col("adjAmtInDifference").rolling_mean(window_size=7),
                      priceImpactSpr=pl.col("priceImpactSpr").rolling_mean(window_size=7),
                     gasCostDifference=pl.col('gasCostDifference').rolling_mean(window_size=7))
        )

In [33]:
costs.write_csv("graphs/data/breakeven_setgas.csv")